# Marathon Data Collection

### 1. Import packages

In [1]:
import pandas as pd
import numpy as np
import math
import time
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import ssl

from datetime import datetime, timedelta

# Ignore SSL certificate errors. Need to also have 'context=ctx' as urlopen parameter.
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

pd.options.mode.chained_assignment = None  # default='warn'

### 2. Pull Race Info and Results

__Choose one__ of the following to collect marathon results from MarathonGuide.com:

- Option A: Save each marathon to an html file, then this code block parses info from all the pages.
- Option B: Use Selenium webdriver to navigate through pages and scrape info (skips deciles to reduce time and requests).
- Option C: Use Selenium webdriver to navigate through pages and scrape info (finds deciles for additional data).

Note: __Option A is preferred__ since it does not hit the site with automated requests. Options B and C use automated requests using Selenium web driver that tend to get blocked eventually, despite added sleep/wait times.

#### Option A: Parse marathon results from locally saved html files ###

In [2]:
# Save each page of MarathonGuide.com you want to parse as an html file in a subfolder named results_html
# Test if results are readable
html = open("results_html/Twin Cities Marathon Race Results 2013.html", "r")
print(html.read()[0:300])




<html>
<head><script type="text/javascript">/* <![CDATA[ */_cf_loadingtexthtml="<img alt=' ' src='/CFIDE/scripts/ajax/resources/cf/images/loading.gif'/>";
_cf_contextpath="";
_cf_ajaxscriptsrc="/CFIDE/scripts/ajax";
_cf_jsonprefix='//';
_cf_websocket_port=8577;
_cf_flash_policy_port=1243;
_cf_cli


In [4]:
# Scrape race results for all race events
# HTML saved locally for each race/year because my selenium scraper got blocked by the site

event_df = pd.DataFrame({'Event ID':[]}).set_index('Event ID')

for file in os.listdir("results_html"):
    if file.endswith(".html"):
        page_source = open(os.path.join("results_html", file), "r").read()
        
        try:
            event_id = re.findall(r'MIDD=(\d+)', page_source)[0]
            # Find the event name, date, city, state, number of finishers, etc.
            event_df.loc[event_id,'Event Name'] = re.findall(
                r'<b>(.+)</b> - Results', page_source)[0]
            event_df.loc[event_id,'Date'] = re.findall(
                r'[A-Z][a-z]+ \d{1,2}, \d{4}', page_source)[1]
            event_df.loc[event_id,['City','State']] = re.findall(
                # r'([\w\-\.\/ ]+), ([A-Z]{2})', page_source)[0] # Doesn't work on "Phoenix, Scottsdale & Tempe"
                r'([^\>]+), ([A-Z]{2})', page_source)[0]
            event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', page_source)[0]
            event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', page_source)[0]
            event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', page_source)[0]

        # In case the event page is missing some data, retrieve in opposite order to get any more data fields you can
        except:
            try:
                event_id = re.findall(r'MIDD=(\d+)', page_source)[0]
                # Find the event name, date, city, state, number of finishers, etc.
                event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                    r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', page_source)[0]
                event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                    r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', page_source)[0]
                event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                    r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', page_source)[0]
                event_df.loc[event_id,['City','State']] = re.findall(
                    r'([^\>]+), ([A-Z]{2})', page_source)[0]
                event_df.loc[event_id,'Date'] = re.findall(
                    r'[A-Z][a-z]+ \d{1,2}, \d{4}', page_source)[2]
                event_df.loc[event_id,'Event Name'] = re.findall(
                    r'<b>(.+)</b> - Results', page_source)[0]
            except:
                # print(event_df.loc[event_id],'\n') # Can print the missing rows if desired
                pass

# Fill in gaps I know about
event_df.loc['57001007',['Male Win','Female Win']] = ('2:21:06','2:51:02') # 2000 St George
event_df.loc['2050116',['Male Win','Female Win']] = ('2:14:50','2:32:27') # 2005 Houston
event_df.loc['67041010',['Male Win','Female Win']] = ('2:06:16','2:23:45') # 2004 Chicago
event_df.loc['41061029',['Male Win','Female Win']] = ('2:21:21','3:00:23') # 2006 Marine Corps
event_df.loc['15060417',['Male Win','Female Win']] = ('2:07:14','2:23:38') # 2006 Boston
event_df.loc['474041128','Time STD'] = '1:01:19' # 2004 Seattle
# Clean up names
event_df.replace({'Chronicle Marathon':'San Francisco Marathon',
                  'The San Francisco Marathon':'San Francisco Marathon',
                  'PORTLANDATHON':'Portland Marathon',
                  'Portland Oregon Marathon':'Portland Marathon',
                  'City of Los Angeles Marathon (L.A. Marathon)':'L.A. Marathon',
                  'Ottawa Marathon (National Capital Race Weekend)':'Ottawa Marathon',
                  "Rock 'n' Roll Marathon":"Rock 'n' Roll San Diego Marathon"
                 }, inplace=True)
# Reduce compound cities to one city (the first in the list):
# Phoenix, Scottsdale & Tempe -> Phoenix
# Minneapolis/St. Paul -> Minneapolis
event_df['City'] = event_df.apply(lambda x: re.split('/|,',x.City)[0], axis=1)

# Save the event results to a csv
event_df = (event_df.sort_values(by=['City','Event ID'],ascending=False)
            .reset_index())
directory = 'results_csvs'
if not os.path.exists(directory):
    print(f'Directory created: {directory}')
    os.mkdir(directory)

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
event_df.to_csv(f'{directory}/all_events_{now}.csv',index=False)
    

In [8]:
# Check the table of marathon results we just made
event_df.describe()

,Event ID,Event Name,Date,City,State,Finishers,Males,Females,Male Win,Female Win,Average Time,Time STD
count,479,479,479,479,479,479,479,479,479,479,479,479
unique,479,25,429,23,17,470,460,461,422,430,461,421
top,41191027,Boston Marathon,"October 7, 2007",Seattle,CA,3218,1755,1597,2:11:56,2:42:15,4:32:02,0:47:39
freq,1,22,3,32,105,2,2,2,5,4,2,4


#### Option B: Parse marathon results with BS4 (skip decile times to reduce page requests)

Warning: you should limit your data scraping using this method due to the high number of automated requests.

In [19]:
# Dictionary where we'll add a df for each event
events_dfs = {}

In [15]:
# List of initial race IDs for scraping MarathonGuide.com 
# Can find these on their website by clicking a particular race and year and looking at the end of the url.
# Each event only needs race ID for one year added, since the race IDs for other years are automatically parsed.
init_ids = {'Boston':'15000417',
    'Chicago':'67001022',
    'New York City':'472001105',
    'LA':'9000305',
    'Marine Corps':'41001022',
    'Honolulu':'480001210',
    'Disney World':'481000108',
    'Rock n Roll San Diego':'27000604',
    'Philadelphia':'479001119',
    'Twin Cities':'58001008',
    'Portland':'38001001',
    'Houston':'2000116',
    'California International':'687001203',
    'St George':'57001007',
    'Grandmas':'42000617',
    'San Francisco':'521050731',
    'Rock N Roll Arizona':''
       }

In [ ]:
# Scrape race results for all race events.
# Increase time.sleep(x) in appropriate spot(s) if needed.

for event, init_id in init_ids.items():
    # Create driver and point to initial url
    init_url = f'http://www.marathonguide.com/results/browse.cfm?MIDD={init_id}'
    driver = webdriver.Safari()
    driver.get(init_url)
    sleeptime = np.random.uniform(26, 29)
    time.sleep(sleeptime)
    
    # Find all the event IDs in the 'a' tag hyperlinks. Each corresponds to a year of the event.
    event_ids = sorted(list(set(re.findall(r'MIDD=(\d+)', driver.page_source))), reverse=True)
    
    # Create df for this event. First column is event ID.
    event_df = (pd.DataFrame({'Event ID':event_ids})
                .set_index('Event ID'))
    
    for event_id in event_ids:
        event_url = f'http://www.marathonguide.com/results/browse.cfm?MIDD={event_id}'
        driver.get(event_url)
        sleeptime = np.random.uniform(11, 14)
        time.sleep(sleeptime)
        
        try:
            # Find the event name, date, city, state, number of finishers, etc.
            event_df.loc[event_id,'Event Name'] = re.findall(
                r'<b>(.+)</b> - Results', driver.page_source)[0]
            event_df.loc[event_id,'Date'] = re.findall(
                r'[A-Z][a-z]+ \d{1,2}, \d{4}', driver.page_source)[1]
            event_df.loc[event_id,['City','State']] = re.findall(
                r'([^\>]+), ([A-Z]{2})', page_source)[0] # Generalized for compound cities
            event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', driver.page_source)[0]
            event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', driver.page_source)[0]
            event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', driver.page_source)[0]
        
        # In case the event page is missing some data, retrieve in opposite order to get any more data fields you can
        except:
            sleeptime = np.random.uniform(21, 24)
            time.sleep(sleeptime)
            try:
                # Find the event name, date, city, state, number of finishers, etc.
                event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                    r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', driver.page_source)[0]
                event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                    r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', driver.page_source)[0]
                event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                    r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', driver.page_source)[0]
                event_df.loc[event_id,['City','State']] = re.findall(
                    r'([^\>]+), ([A-Z]{2})', page_source)[0] # Generalized for compound cities
                event_df.loc[event_id,'Date'] = re.findall(
                    r'[A-Z][a-z]+ \d{1,2}, \d{4}', driver.page_source)[1]
                event_df.loc[event_id,'Event Name'] = re.findall(
                    r'<b>(.+)</b> - Results', driver.page_source)[0]
            except:
                # print(event_df.loc[event_id]) # Can print the missing rows if desired
                pass

    
    # Add this event_df to the dictionary of events_dfs
    events_dfs[event] = event_df
    # Save the event results to a csv
    directory = 'events_csvs'
    if not os.path.exists(directory):
        print(f'Directory created: {directory}')
        os.mkdir(directory)

    now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
    event_df.to_csv(f'{directory}/{event}_{now}.csv',index=True)
    
    driver.close()

In [ ]:
# Combine all events (all marathons, all years) into a larger df
combined_df = pd.concat(events_dfs.values())

In [ ]:
# Save the combined results to a csv
directory = 'results_csvs'
if not os.path.exists(directory):
    print(f'Directory created: {directory}')
    os.mkdir(directory)

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
combined_df.to_csv(f'{directory}/race_results_{now}.csv',index=True)

#### Option C: Scrape marathon results with Selenium and BS4 (includes decile times)

Warning: you should limit your data scraping using this method due to the high number of automated requests.

In [10]:
# Dictionary where we'll add a df for each event
events_dfs = {}

In [11]:
# List of initial race IDs for scraping MarathonGuide.com 
# Can find these on their website by clicking a particular race and year and looking at the end of the url.
# Each event only needs race ID for one year added, since the race IDs for other years are automatically parsed.
init_ids = {'Boston':'15000417',
        'Chicago':'67001022',
        'New York City':'472001105'
       }

In [ ]:
# Scrape race results for all race events.
# Increase time.sleep(x) in appropriate spot(s) if needed.

for event, init_id in init_ids.items():
    # Create driver and point to initial url
    init_url = f'http://www.marathonguide.com/results/browse.cfm?MIDD={init_id}'
    driver = webdriver.Safari()
    driver.get(init_url)
    sleeptime = np.random.uniform(6, 9)
    time.sleep(sleeptime)
    
    # Find all the event IDs in the 'a' tag hyperlinks. Each corresponds to a year of the event.
    event_ids = sorted(list(set(re.findall(r'MIDD=(\d+)', driver.page_source))), reverse=True)
    
    # Create df for this event. First column is event ID.
    event_df = (pd.DataFrame({'Event ID':event_ids})
                .set_index('Event ID'))
    
    for event_id in event_ids:
        event_url = f'http://www.marathonguide.com/results/browse.cfm?MIDD={event_id}'
        driver.get(event_url)
        sleeptime = np.random.uniform(6, 9)
        time.sleep(sleeptime)
        
        try:
            # Find the event name, date, city, state, number of finishers, etc.
            event_df.loc[event_id,'Event Name'] = re.findall(
                r'<b>(.+)</b> - Results', driver.page_source)[0]
            event_df.loc[event_id,'Date'] = re.findall(
                r'[A-Z][a-z]+ \d{1,2}, \d{4}', driver.page_source)[1]
            event_df.loc[event_id,['City','State']] = re.findall(
                r'([^\>]+), ([A-Z]{2})', page_source)[0] # Generalized for compound cities
            event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', driver.page_source)[0]
            event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', driver.page_source)[0]
            event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', driver.page_source)[0]
        
        # In case the event page is missing some data, only get deciles if we have finisher count
        except:
            print(event_df.loc[event_id])
            if np.isnan(float(event_df.loc[event_id,'Finishers'])): continue
            sleeptime = np.random.uniform(6, 9)
            time.sleep(sleeptime)
            pass
        
        # Calculate Decile Places
        finishers = float(event_df.loc[event_id,'Finishers'])
        decile_places = [str(int(decile)) for decile in np.linspace(1,finishers,11)]
        event_df.loc[event_id,['D0 Place','D1 Place','D2 Place','D3 Place','D4 Place','D5 Place',
                              'D6 Place','D7 Place','D8 Place','D9 Place', 'D10 Place']
                    ] = decile_places
        
        # Names for cols for decile times we'll iterate over
        decile_cols = ['D0','D1','D2','D3','D4','D5','D6','D7','D8','D9','D10']
        
        # Find the finish time for each decile
        for finish_place, decile in zip(decile_places,decile_cols):
            try:
                # Find the dropdown/Select elements for finisher place range/options
                selector = driver.find_element(by=By.XPATH, value="//select[@name='RaceRange']")
                all_options = selector.find_elements(by=By.TAG_NAME, value="option")
                results_page = int((int(finish_place)+99)/100)
                option = all_options[results_page]
                submit_button = driver.find_element(by=By.XPATH, value="//input[@name='SubmitButton']")

                # Load the results page
                selector.click()
                option.click()
                submit_button.click()
                sleeptime = np.random.uniform(6, 9)
                time.sleep(sleeptime)

                # The results page contains several levels of nested tables.
                # Depending on the particular page, here are possible paths
                try:
                    soup = BeautifulSoup(driver.page_source)
                    table1 = soup.find_all('table')[9]
                    table2 = table1.find_all('table')[3]
                    table3 = table2.find_all('table')[0]
                    table4 = table3.find_all('table')[0]
                    table5 = table4.find_all('table')[0]
                except:
                    soup = BeautifulSoup(driver.page_source)
                    table1 = soup.find_all('table')[11]
                    table2 = table1.find_all('table')[3]
                    table3 = table2.find_all('table')[0]
                    table4 = table3.find_all('table')[0]
                    table5 = table4.find_all('table')[0]

                # Convert the table into pandas df and clean
                page_df = pd.read_html(str(table5))[0].iloc[2:]
                page_df.columns = page_df.iloc[0]
                page_df = page_df.drop(index = 2)
                # Account for different column naming per event
                if 'Net Time' not in page_df.columns:
                    page_df = page_df.rename(columns={'Time':'Net Time'})

                # Get the finish time for the desired finish place and update event_df
                finish_time = (page_df[page_df['OverAllPlace'] == finish_place]
                               .loc[:,'Net Time']
                               .iloc[0])
                event_df.loc[event_id,decile] = finish_time
                
                # Return to event page to prep for next decile
                driver.get(event_url)
                sleeptime = np.random.uniform(6, 9)
                time.sleep(sleeptime)
                
            # If parsing a decile time fails:
            # print data, leave decile time as NaN, and continue to remaining deciles
            except:
                print(event_df.loc[event_id,['Event Name','Date']])
                print(f'    {decile}: {finish_place}\n')
                sleeptime = np.random.uniform(6, 9)
                time.sleep(sleeptime)
                pass
            
                # Return to event page to prep for next decile
                driver.get(event_url)
                sleeptime = np.random.uniform(6, 9)
                time.sleep(sleeptime)
    
    # Add this event_df to the dictionary of events_dfs
    events_dfs[event] = event_df
    # Save the event results to a csv
    directory = 'events_csvs'
    if not os.path.exists(directory):
        print(f'Directory created: {directory}')
        os.mkdir(directory)

    now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
    event_df.to_csv(f'{directory}/{event}_{now}.csv',index=True)
    
    driver.close()

In [ ]:
# Check race results csvs for missing data and try pulling just those rows again

for file in os.listdir("results_csvs"):
    if file.endswith(".csv"):
        print(os.path.join("results_csvs", file))
        
        # Open a df and get the index of all rows with one or more missing cells (NaN)
        event_df = (pd.read_csv(os.path.join("results_csvs", file))
                   .set_index('Event ID'))
        nan_rows = event_df[event_df.isnull().T.any()]
        print(nan_rows.index)
        
        for event_id in nan_rows.index:
            event_url = f'http://webcache.googleusercontent.com/search?q=cache:http://www.marathonguide.com/results/browse.cfm?MIDD={event_id}'
            # Create driver and point to url
            driver = webdriver.Safari()
            driver.get(event_url)
            sleeptime = np.random.uniform(6, 9)
            time.sleep(sleeptime)

            try:
                # Find the event name, date, city, state, number of finishers, etc.
                event_df.loc[event_id,'Event Name'] = re.findall(
                    r'<b>(.+)</b> - Results', driver.page_source)[0]
                event_df.loc[event_id,'Date'] = re.findall(
                    r'[A-Z][a-z]+ \d{1,2}, \d{4}', driver.page_source)[1]
                event_df.loc[event_id,['City','State']] = re.findall(
                    r'([\w\- ]+), ([A-Z]{2})', driver.page_source)[0]
                event_df.loc[event_id,['Finishers','Males','Females']] = re.findall(
                    r'Finishers: (\d+), Males - (\d+) , Females - (\d+)', driver.page_source)[0]
                event_df.loc[event_id,['Male Win','Female Win']] = re.findall(
                    r'Male Winner: ([\d:]+) \| Female Winner: ([\d:]+)', driver.page_source)[0]
                event_df.loc[event_id,['Average Time','Time STD']] = re.findall(
                    r'Average Finish Time: ([\d:]+) \| STD: ([\d:]+)', driver.page_source)[0]

            # In case the event page is missing some data, only get deciles if we have finisher count
            except:
                print(event_df.loc[event_id])
                if np.isnan(float(event_df.loc[event_id,'Finishers'])): continue
                sleeptime = np.random.uniform(6, 9)
                time.sleep(sleeptime)
                pass

            # Calculate Decile Places
            finishers = float(event_df.loc[event_id,'Finishers'])
            decile_places = [str(int(decile)) for decile in np.linspace(1,finishers,11)]
            event_df.loc[event_id,['D0 Place','D1 Place','D2 Place','D3 Place','D4 Place','D5 Place',
                                  'D6 Place','D7 Place','D8 Place','D9 Place', 'D10 Place']
                        ] = decile_places

            # Names for cols for decile times we'll iterate over
            decile_cols = ['D0','D1','D2','D3','D4','D5','D6','D7','D8','D9','D10']

            # Find the finish time for each decile
            for finish_place, decile in zip(decile_places,decile_cols): # [0:3] REMOVE these indices to get all deciles
                try:
                    # Find the dropdown/Select elements for finisher place range/options
                    selector = driver.find_element(by=By.XPATH, value="//select[@name='RaceRange']")
                    all_options = selector.find_elements(by=By.TAG_NAME, value="option")
                    results_page = int((int(finish_place)+99)/100)
                    option = all_options[results_page]
                    submit_button = driver.find_element(by=By.XPATH, value="//input[@name='SubmitButton']")

                    # Load the results page
                    selector.click()
                    option.click()
                    submit_button.click()
                    sleeptime = np.random.uniform(6, 9)
                    time.sleep(sleeptime)

                    # The results page contains several levels of nested tables.
                    # Depending on the particular page, here are possible paths
                    try:
                        soup = BeautifulSoup(driver.page_source)
                        table1 = soup.find_all('table')[9]
                        table2 = table1.find_all('table')[3]
                        table3 = table2.find_all('table')[0]
                        table4 = table3.find_all('table')[0]
                        table5 = table4.find_all('table')[0]
                    except:
                        soup = BeautifulSoup(driver.page_source)
                        table1 = soup.find_all('table')[11]
                        table2 = table1.find_all('table')[3]
                        table3 = table2.find_all('table')[0]
                        table4 = table3.find_all('table')[0]
                        table5 = table4.find_all('table')[0]

                    # Convert the table into pandas df and clean
                    page_df = pd.read_html(str(table5))[0].iloc[2:]
                    page_df.columns = page_df.iloc[0]
                    page_df = page_df.drop(index = 2)
                    # Account for different column naming per event
                    if 'Net Time' not in page_df.columns:
                        page_df = page_df.rename(columns={'Time':'Net Time'})

                    # Get the finish time for the desired finish place and update event_df
                    finish_time = (page_df[page_df['OverAllPlace'] == finish_place]
                                   .loc[:,'Net Time']
                                   .iloc[0])
                    event_df.loc[event_id,decile] = finish_time

                    # Return to event page to prep for next decile
                    driver.get(event_url)
                    sleeptime = np.random.uniform(6, 9)
                    time.sleep(sleeptime)

                # If parsing a decile time fails:
                # print data, leave decile time as NaN, and continue to remaining deciles
                except:
                    print(event_df.loc[event_id,['Event Name','Date']])
                    print(f'    {decile}: {finish_place}\n')
                    sleeptime = np.random.uniform(6, 9)
                    time.sleep(sleeptime)
                    pass

                    # Return to event page to prep for next decile
                    driver.get(event_url)
                    sleeptime = np.random.uniform(6, 9)
                    time.sleep(sleeptime)
            driver.close()

        # Update this event_df in the dictionary of events_dfs
        events_dfs[event] = event_df
        # Save the event results to a csv
        directory = 'events_csvs_cleaned'
        if not os.path.exists(directory):
            print(f'Directory created: {directory}')
            os.mkdir(directory)

        event = event_df.loc[:,'Event Name'].iloc[0]
        now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
        event_df.to_csv(f'{directory}/{event}_{now}.csv',index=True)

In [ ]:
# Combine all events (all marathons, all years) into a larger df
combined_df = pd.concat(events_dfs.values())

In [ ]:
# Save the combined results to a csv
directory = 'results_csvs'
if not os.path.exists(directory):
    print(f'Directory created: {directory}')
    os.mkdir(directory)

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
combined_df.to_csv(f'{directory}/race_results_{now}.csv',index=True)

### 3. Get elevation data

In [16]:
print(sorted(event_df['Event Name'].unique()))

elevation_columns = ['Event Name','Elev Gain','Elev Loss','Elev Max','Elev Min'] # units of feet

# Data from https://findmymarathon.com/ 
elevation_metrics = {'Big Sur International Marathon':[1654,1949,578,8],
                     'Boston Marathon':[815,1275,470,10],
                     'California International Marathon':[663,1003,359,19],
                     'Chicago Marathon':[243,242,612,580],
                     'Disney World Marathon':[351,354,105,75],
                     "Grandma's Marathon":[471,581,727,603],
                     'Honolulu Marathon':[457,455,126,3],
                     'Houston Marathon':[225,222,84,27],
                     'L.A. Marathon':[943,1169,566,203],
                     'Marine Corps Marathon':[630,593,232,2],
                     'New York City Marathon':[810,824,260,7],
                     'Philadelphia Marathon':[846,820,148,5],
                     'Portland Marathon':[873,873,178,30],
                     "Rock 'n' Roll Arizona Marathon":[509,516,1277,1151],
                     "Rock 'n' Roll San Diego Marathon":[798,1031,403,8],
                     "Rock 'n' Roll Seattle Marathon":[1149,1155,404,17],
                     'San Francisco Marathon':[1365,1364,306,5],
                     'Seattle Marathon':[931,929,192,21],
                     'St. George Marathon':[500,3057,5244,2685],
                     'Twin Cities Marathon':[576,495,950,788]
                     }

elev_df = pd.DataFrame(elevation_metrics).T.reset_index()
elev_df.columns = elevation_columns

# Save a copy of the elevation data
directory = 'elevation_csvs'
if not os.path.exists(directory):
    print(f'Directory created: {directory}')
    os.mkdir(directory)

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
elev_df.to_csv(f'{directory}/elevations_{now}.csv',index=False)

['Big Sur International Marathon', 'Boston Marathon', 'California International Marathon', 'Chicago Marathon', 'Disney World Marathon', "Grandma's Marathon", 'Honolulu Marathon', 'Houston Marathon', 'L.A. Marathon', 'Marine Corps Marathon', 'Montreal International Marathon', 'New York City Marathon', 'Ottawa Marathon', 'Philadelphia Marathon', 'Portland Marathon', "Rock 'n' Roll Arizona Marathon", "Rock 'n' Roll Montreal Marathon", "Rock 'n' Roll San Diego Marathon", "Rock 'n' Roll Seattle Marathon", 'San Francisco Marathon', 'Seattle Marathon', 'St. George Marathon', 'Toronto Waterfront Marathon', 'Twin Cities Marathon', 'Vancouver International Marathon']


In [17]:
elev_df

,Event Name,Elev Gain,Elev Loss,Elev Max,Elev Min
0,Big Sur International Marathon,1654,1949,578,8
1,Boston Marathon,815,1275,470,10
2,California International Marathon,663,1003,359,19
3,Chicago Marathon,243,242,612,580
4,Disney World Marathon,351,354,105,75
5,Grandma's Marathon,471,581,727,603
6,Honolulu Marathon,457,455,126,3
7,Houston Marathon,225,222,84,27
8,L.A. Marathon,943,1169,566,203
9,Marine Corps Marathon,630,593,232,2


### 4. Pull Weather Data

1. Define a couple of useful functions
2. Open marathon results df to get cities and dates
3. Pull weather from NOAA NCEI
4. Save Weather df to csv

In [19]:
# Source of Weather Data:

# NCEI Global Summary of the Day:
# https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00516

# Data types: https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt
# TEMP - Mean temperature (.1 Fahrenheit)
# DEWP - Mean dew point (.1 Fahrenheit)
# SLP - Mean sea level pressure (.1 mb)
# STP - Mean station pressure (.1 mb)
# VISIB - Mean visibility (.1 miles)
# WDSP – Mean wind speed (.1 knots)
# MXSPD - Maximum sustained wind speed (.1 knots)
# GUST - Maximum wind gust (.1 knots)
# MAX - Maximum temperature (.1 Fahrenheit)
# MIN - Minimum temperature (.1 Fahrenheit)
# PRCP - Precipitation amount (.01 inches)
# SNDP - Snow depth (.1 inches)
# FRSHTT – Indicator for occurrence of:
#                               Fog
#                               Rain or Drizzle
#                               Snow or Ice Pellets
#                               Hail
#                               Thunder
#                               Tornado/Funnel Cloud

# Find Stations: https://www.ncei.noaa.gov/maps/daily/?layers=0001
# Generally used closest airport since data usually covers full range of years

In [20]:
# Weather Stations
station_ids = {'Boston':['72509014739'],
               'Carmel':['72491523259','72491599999','72491523245'],
               'Chicago':['72530094846'],
               'Duluth':['72745014913'],
               'Honololu':['91182022521'],
               'Houston':['72244012918','72243012960'],
               'Los Angeles':['72288593197','72288599999','72287493134'],
               'Minneapolis':['72658014922'],
               'Montreal':['71371099999','71612099999','71627099999','71627094792'],
               'New York City':['72503014732', '99999914732'],
               'Orlando':['72205012815'],
               'Ottawa':['71628099999','71627999999'],
               'Philadelphia':['72408013739'],
               'Phoenix':['72278023183'],
               'Portland':['72698024229'],
               'Sacramento':['72483993225','72483999999','72483323206','72483399999'],
               'San Diego':['72290023188'],
               'San Francisco':['72494023234'],
               'Seattle':['72793524234','72793024233'],
               'St. George':['72475423186','72475499999','72092299999','72475593129'],
               'Toronto':['71265099999','71624099999'],
               'Vancouver':['71892099999','71784099999'],
               'Washington':['72405013743'],
               'Berlin':['10382099999'],'London':['3772099999'],'Tokyo':['47662099999']}

# Weather data types we'll keep
weather_cols = ['STATION','DATE','LATITUDE','LONGITUDE','ELEVATION',
                'TEMP','MAX','MIN','PRCP','DEWP',
                'WDSP','SLP','STP','VISIB']

# Main function for getting NCEI weather data
# Note that the NOAA files are downloaded and saved to the local drive
# This makes future runs faster for you and reduces hits on the database server
def get_weather(date, city):
    try:
        stations = station_ids[city]
        year = date.year
        directory = f'weather_csvs/{city}/'
        if not os.path.exists(directory):
            print(f'Created directory {directory}.')
            os.mkdir(directory)
        # Try multiple station_ids for the same location since ids are occasionally updated
        for station_id in station_ids[city]:
            file_name = f'{station_id}_{year}.csv'
            try:
                try: # Check if file saved locally from previous run
                    temp_df = pd.read_csv(directory+file_name)
                except:
                    url = f'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/{year}/{station_id}.csv'
                    temp_df = pd.read_csv(url)
                    temp_df.to_csv(f'{directory}{file_name}',index=False)
                    print(f'Retrieved {file_name} online and saved to local drive folder {directory}.')
                    sleeptime = np.random.uniform(1, 2)
                    time.sleep(sleeptime)
                temp_df['DATE'] = pd.to_datetime(temp_df['DATE'], format="%Y-%m-%d")
                weather_data = temp_df[temp_df['DATE']==date]
                weather_data = weather_data[weather_cols].squeeze().tolist()
                break
            except: continue # on to check the next station in this city
            
        temp_df['DATE'] = pd.to_datetime(temp_df['DATE'], format="%Y-%m-%d")
        weather_data = temp_df[temp_df['DATE']==date]
        weather_data = weather_data[weather_cols].squeeze().tolist()
    except: # If function fails, get empty weather data (NCEI records don't go back far enough)
        weather_data = pd.DataFrame(columns = weather_cols,
                                    index = [0]).squeeze().tolist()
    return weather_data

# Secondary weather function to calculate relative humidity based on temperature and dewpoint (degrees F)
def calc_humidity(temp_f,dewpoint_f):
    # Convert temps from F to C
    T = (temp_f-32)*5/9
    DP = (dewpoint_f-32)*5/9
    
    # Save constants
    a = 17.625
    b = 243.04
    
    # Formula from https://bmcnoldy.rsmas.miami.edu/Humidity.html
    RH = 100*(math.exp((a*DP)/(b+DP))/math.exp((a*T)/(b+T)))
    return RH

In [21]:
# Test get_weather
get_weather(pd.to_datetime('2005-05-01', format="%Y-%m-%d"), 'Vancouver')

[71784099999,
 Timestamp('2005-05-01 00:00:00'),
 49.35,
 -123.2,
 168.0,
 54.6,
 66.2,
 46.4,
 0.07,
 49.1,
 1.7,
 1020.8,
 0.5,
 999.9]

In [22]:
# Load marathon results
directory = 'results_csvs'
newest_results = [file for file in sorted(os.listdir(directory)) if file.endswith('.csv')][-1]
event_df = pd.read_csv(os.path.join(directory, newest_results))
# print(os.path.join(directory, newest_results))
event_df['Date'] = pd.to_datetime(event_df['Date'])

# Pull the weather data according to Date and City
event_df[weather_cols] = event_df.apply(lambda x: get_weather(x.Date, x.City), axis=1, result_type='expand')
event_df['RELHUM'] = event_df.apply(lambda x: calc_humidity(x.TEMP, x.DEWP), axis=1)
    
weather_df = (event_df[['Date', 'City', 'State', 'STATION', 
                       'LATITUDE', 'LONGITUDE', 'ELEVATION', 
                       'TEMP', 'MAX', 'MIN', 'PRCP', 'DEWP', 'WDSP', 
                       'SLP', 'STP', 'VISIB', 'RELHUM']]
              .sort_values(['City','Date']))

weather_df['PRCP'].replace({99.99:0}, inplace=True) # NOAA database uses all 9's for unrecorded data

# Save a copy of the weather data
directory = 'weather_table_csvs'
if not os.path.exists(directory):
    os.mkdir(directory)
    print(f'Directory created: {directory}')

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
weather_df.to_csv(f'{directory}/weather_{now}.csv',index=False)

In [23]:
# Check the output
weather_df.describe()

# In this NOAA database, if data item is unrecorded/missing then all digits will be 9's.
# You may want to drop rows with bad data.
# We'll drop entire columns for SLP and STP since those have lots of missing data.

,STATION,LATITUDE,LONGITUDE,ELEVATION,TEMP,MAX,MIN,PRCP,DEWP,WDSP,SLP,STP,VISIB,RELHUM
count,4.790000e+02,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000
mean,7.316054e+10,39.247881,-102.064404,111.088079,57.139040,67.765762,48.072234,0.070188,44.995825,6.877035,1336.602505,404.716701,13.972025,66.331074
std,3.881581e+09,6.874670,23.069471,213.862844,9.624765,10.903297,9.934307,0.204460,11.590381,3.400088,1663.580462,476.808757,63.986119,16.776645
min,7.126510e+10,21.324000,-157.939460,1.900000,31.100000,39.000000,23.000000,0.000000,4.600000,1.100000,989.700000,0.300000,0.600000,14.356391
25%,7.227802e+10,34.017000,-121.845300,3.700000,50.700000,60.100000,41.000000,0.000000,36.700000,4.400000,1014.100000,13.800000,9.200000,56.789433
50%,7.248399e+10,39.873270,-95.360000,13.400000,57.500000,68.000000,48.000000,0.000000,46.100000,6.200000,1017.900000,22.500000,9.900000,69.686268
75%,7.253009e+10,45.322500,-79.383333,113.395000,64.050000,75.900000,55.000000,0.030000,53.550000,9.000000,1022.600000,994.450000,10.000000,77.992114
max,9.118202e+10,49.350000,-71.009700,1702.600000,80.000000,95.000000,73.000000,2.240000,68.900000,23.300000,9999.900000,999.900000,999.900000,97.841783


In [28]:
# Check the weather data grouped by city if interested. Could look at PRCP, TEMP, MIN, MAX, etc.
weather_df.groupby(by=['City']).agg({"PRCP":[np.median,np.mean,np.max]}).sort_values(('PRCP','amax'))

PRCP                
              median      mean  amax
City                                
St. George     0.000  0.000000  0.00
San Francisco  0.000  0.000000  0.00
Phoenix        0.000  0.001765  0.03
San Diego      0.000  0.001905  0.04
Ottawa         0.000  0.021579  0.11
Carmel         0.000  0.014762  0.16
Los Angeles    0.000  0.014091  0.22
New York City  0.000  0.032000  0.23
Orlando        0.000  0.048182  0.41
Chicago        0.000  0.050476  0.41
Vancouver      0.020  0.084737  0.46
Seattle        0.015  0.077813  0.47
Minneapolis    0.000  0.047143  0.49
Houston        0.000  0.072273  0.52
Portland       0.010  0.094762  0.66
Toronto        0.015  0.135000  0.69
Sacramento     0.005  0.122500  0.76
Philadelphia   0.000  0.067143  0.86
Montreal       0.000  0.082500  0.89
Washington     0.005  0.180000  0.90
Duluth         0.020  0.177143  1.03
Honololu       0.000  0.139048  2.00
Boston         0.000  0.147273  2.24

### 5. Inner Join Race results, elevation, and weather

In [29]:
# Load marathon results
directory = 'results_csvs'
newest_results = [file for file in sorted(os.listdir(directory)) if file.endswith('.csv')][-1]
results_df = pd.read_csv(os.path.join(directory, newest_results))
results_df['Date'] = pd.to_datetime(results_df['Date'])
time_cols = ['Male Win','Female Win','Average Time','Time STD']
results_df[time_cols] = (results_df[time_cols].apply(pd.to_timedelta))
# Add column of sex ratio
results_df['Percent Female'] = 100*results_df['Females']/(results_df['Females']+results_df['Males'])
# Drop Boston 2013 because course was close mid-race for emergency
Bos_2013 = results_df[(results_df['Event Name'] == 'Boston Marathon') & (results_df['Date'].dt.year == 2013)].index
results_df = results_df.drop(Bos_2013)

# Load elevation data
directory = 'elevation_csvs'
newest_elevation = [file for file in sorted(os.listdir(directory)) if file.endswith('.csv')][-1]
elevation_df = pd.read_csv(os.path.join(directory, newest_elevation))

# Load weather data
directory = 'weather_table_csvs'
newest_weather = [file for file in sorted(os.listdir(directory)) if file.endswith('.csv')][-1]
weather_df = pd.read_csv(os.path.join(directory, newest_weather))
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

# Join all three data sets
full_df = (results_df
           .merge(elevation_df, how='inner', on='Event Name')
           .merge(weather_df, how='inner', on=['Date','City','State'])
           .sort_values(['Event Name','Date']))

In [30]:
# Check Output
full_df

,Event ID,Event Name,Date,City,State,Finishers,Males,Females,Male Win,Female Win,...,TEMP,MAX,MIN,PRCP,DEWP,WDSP,SLP,STP,VISIB,RELHUM
384,18000430,Big Sur International Marathon,2000-04-30,Carmel,CA,2407,1535,872,0 days 02:27:06,0 days 02:46:53,...,53.6,63.0,42.1,0.00,45.7,5.4,1022.0,999.9,10.0,74.520777
383,18010429,Big Sur International Marathon,2001-04-29,Carmel,CA,2560,1548,1012,0 days 02:25:38,0 days 02:46:41,...,52.8,60.8,46.4,0.00,44.8,5.8,1021.2,999.9,10.0,74.161050
382,18020428,Big Sur International Marathon,2002-04-28,Carmel,CA,2343,1492,851,0 days 02:18:05,0 days 02:51:10,...,49.8,60.1,41.0,0.08,41.1,5.9,1021.1,999.9,10.0,71.896619
381,18030427,Big Sur International Marathon,2003-04-27,Carmel,CA,2820,1755,1065,0 days 02:19:59,0 days 02:47:11,...,54.2,63.0,44.1,0.00,41.1,7.1,1016.9,999.9,10.0,61.137048
380,18040425,Big Sur International Marathon,2004-04-25,Carmel,CA,2858,1815,1043,0 days 02:26:19,0 days 03:10:06,...,62.5,80.6,50.0,0.00,48.0,2.3,1016.7,999.9,10.0,59.046912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,58161009,Twin Cities Marathon,2016-10-09,Minneapolis,MN,8551,4714,3837,0 days 02:08:51,0 days 02:30:01,...,46.0,60.1,36.0,0.00,35.9,3.9,1027.8,996.7,8.9,67.650899
239,58171001,Twin Cities Marathon,2017-10-01,Minneapolis,MN,7518,4100,3418,0 days 02:11:53,0 days 02:30:25,...,60.0,69.1,46.0,0.06,48.6,11.1,1017.2,986.4,9.5,65.976075
238,58181007,Twin Cities Marathon,2018-10-07,Minneapolis,MN,7157,3972,3185,0 days 02:11:58,0 days 02:33:04,...,47.0,52.0,44.1,0.00,41.7,7.1,1024.9,993.4,9.1,81.711865
237,58191006,Twin Cities Marathon,2019-10-06,Minneapolis,MN,6739,3849,2890,0 days 02:12:23,0 days 02:31:29,...,54.0,64.9,46.9,0.49,41.9,11.7,1012.6,982.0,9.9,63.516169


In [31]:
# Save a copy of the full data set
directory = 'full_df_csvs'
if not os.path.exists(directory):
    os.mkdir(directory)
    print(f'Directory created: {directory}')

now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
full_df.to_csv(f'{directory}/full_{now}.csv',index=False)